# This notebook provides the functionality to build, train, and test a CNN for predicting mosquito age, grouped age, species, and status.

## Structure:
* Import packages to be used.
* Load mosquito data.
* Define fucntions for plotting, visualisation, and logging.
* Define a function to build the CNN.
* Define a function to train the CNN.
* Main section to organise data, define the CNN, and call the building and training of the CNN.


In [1]:
import pylab as pl
import datetime
import pandas as pd
import itertools
from itertools import cycle
import pickle
import random as rn
import os
from time import time
from tqdm import tqdm

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib

font = {'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils import resample

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras import layers, metrics
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import model_from_json, load_model
from keras.regularizers import *
from keras.callbacks import CSVLogger
from keras import backend as K

Using TensorFlow backend.


In [2]:
# rand_seed = np.random.randint(low=0, high=100)
rand_seed = 16
print(rand_seed)

16


In [3]:
os.environ['PYTHONHASHSEED'] = '0'

## The below is necessary for starting Numpy generated random numbers in a well-defined initial state.
np.random.seed(rand_seed)

## The below is necessary for starting core Python generated random numbers in a well-defined state.
rn.seed(12345)

## Force TensorFlow to use single thread.
## Multiple threads are a potential source of
## non-reproducible results.
## For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res
# session_conf = tf.ConfigProto(device_count = {'GPU':0}, intra_op_parallelism_threads=4) #session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# session_conf = tf.ConfigProto(device_count = {'GPU':0}) #session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#session_conf.gpu_options.per_process_gpu_memory_fraction = 0.5
## The below tf.set_random_seed() will make random number generation
## in the TensorFlow backend have a well-defined initial state.
## For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed
tf.set_random_seed(1234)

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.35)

sess = tf.Session(graph=tf.get_default_graph(), config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

## Load the data

The data file is created using Loco Mosquito:
https://github.com/magonji/MIMI-project/blob/master/Loco%20mosquito%204.0.ipynb

### The data file has headings: Species - Status - RearCnd - Age - Country- Frequencies

In [4]:
class data_loader_class():
    def __init__(self, valid_perc):
        df = pd.read_csv("/home/josh/Documents/Mosquito_Project/MIMI-Analysis/Data/mosquitoes_country_LM_5_0.dat", '\t')
        df.head(10)

        df['AgeGroup'] = 0
        df['AgeGroup'] = np.where(df['Age']>10, 2, np.where(df['Age']>4, 1, 0))

        df_vf = df[df['RearCnd']=='VF']
        df_vf = df_vf[df_vf['Status']=='UN']
        df = df[df['RearCnd']!='VF']
        df = df[df['Status']!='UN']
        df_l = df[df['RearCnd']=='TL']
        df_l_g = df_l[df_l['Country']=='S']
        
        df_l_g_a = df_l_g[df_l_g['Species']=='AA']
        age_counts = df_l_g_a.groupby('AgeGroup').size()
        df_l_g_g = df_l_g[df_l_g['Species']=='AG']
        age_counts = df_l_g_g.groupby('AgeGroup').size()
        df_l_g_c = df_l_g[df_l_g['Species']=='AC']
        age_counts = df_l_g_c.groupby('AgeGroup').size()
        df_l_t = df_l[df_l['Country']=='T']
        df_l_t_a = df_l_t[df_l_t['Species']=='AA']
        age_counts = df_l_t_a.groupby('AgeGroup').size()
        df_l_t_g = df_l_t[df_l_t['Species']=='AG']
        age_counts = df_l_t_g.groupby('AgeGroup').size()
        df_l_b = df_l[df_l['Country']=='B']
        df_l_b_g = df_l_b[df_l_b['Species']=='AG']
        age_counts = df_l_b_g.groupby('AgeGroup').size()
        df_l_b_c = df_l_b[df_l_b['Species']=='AC']
        age_counts = df_l_b_c.groupby('AgeGroup').size()
        df_f = df[df['RearCnd']=='TF']
        df_f_t = df_f[df_f['Country']=='T']
        df_f_t_a = df_f_t[df_f_t['Species']=='AA']
        # df_f_t_g = df_f_t[df_f_t['Species']=='AG'] #There isn't any
        df_f_b = df_f[df_f['Country']=='B']
        df_f_b_g = df_f_b[df_f_b['Species']=='AG']
        age_counts = df_f_b_g.groupby('AgeGroup').size()
        df_f_b_c = df_f_b[df_f_b['Species']=='AC']
        age_counts = df_f_b_c.groupby('AgeGroup').size()
        df_vf_t = df_vf[df_vf['Country']=='T']
        df_vf_t_a = df_vf_t[df_vf_t['Species']=='AA']
        age_counts = df_vf_t_a.groupby('AgeGroup').size()
#         print(age_counts)
        df_vf_t_g = df_vf_t[df_vf_t['Species']=='AG']
        age_counts = df_vf_t_g.groupby('AgeGroup').size()
#         print(age_counts)
        df_vf_b = df_vf[df_vf['Country']=='B']
        df_vf_b_g = df_vf_b[df_vf_b['Species']=='AG']
        age_counts = df_vf_b_g.groupby('AgeGroup').size()
#         print(age_counts)
        df_vf_b_c = df_vf_b[df_vf_b['Species']=='AC']
        age_counts = df_vf_b_c.groupby('AgeGroup').size()
#         print(age_counts)

        VF_size_t = len(df_vf_t)
        VF_size_b = len(df_vf_b)
#         print('validation size tanzania : {}'.format(VF_size_t))
#         print('validation size bobo : {}'.format(VF_size_b))
        val_group_size_t = int(((((valid_perc*VF_size_t)/2)/3))) #/2 (t/b) /3 (age groups)
        val_group_size_b = int(((((valid_perc*VF_size_b)/2)/3)))
#         print('validation size for testing tanzania : {}'.format(val_group_size_t))
#         print('validation size for testing bobo : {}'.format(val_group_size_b))

        size_inc = 400
        for age in range(3):
            df_temp = df_l_t_a[df_l_t_a['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            if age == 0:
                df_train = df_temp.iloc[index_df_temp_inc]
        #         df_test = df_temp.iloc[index_df_temp_not_inc]
            else:
                df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
        #         df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        for age in range(3):
            df_temp = df_l_t_g[df_l_t_g['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
        #     df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        size_inc = 400
        for age in range(3):
            df_temp = df_l_b_g[df_l_b_g['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
        #     df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        for age in range(3):
            df_temp = df_l_b_c[df_l_b_c['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
        #     df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        size_inc = 300
        for age in range(3):
            df_temp = df_f_t_a[df_f_t_a['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
        #     df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        for age in range(3):
            df_temp = df_f_b_g[df_f_b_g['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
        #     df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        size_inc = 300
        for age in range(3):
            df_temp = df_f_b_c[df_f_b_c['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
        #     df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        size_inc = val_group_size_t
        for age in range(3):
            df_temp = df_vf_t_a[df_vf_t_a['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            if len(size_df_temp) < size_inc:
                print('Warning Tanzania Arabiensis VF group {} smaller than amount requested'.format(age))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
            if age == 0:
                df_test = df_temp.iloc[index_df_temp_not_inc]
            else:
                df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        for age in range(3):
            df_temp = df_vf_t_g[df_vf_t_g['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            if len(size_df_temp) < size_inc:
                print('Warning Tanzania Gambie VF group {} smaller than amount requested'.format(age))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
            df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        size_inc = val_group_size_b
        for age in range(3):
            df_temp = df_vf_b_g[df_vf_b_g['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            if len(size_df_temp) < size_inc:
                print('Warning Bobo Gambie VF group {} smaller than amount requested'.format(age))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
            df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])
        for age in range(3):
            df_temp = df_vf_b_c[df_vf_b_c['AgeGroup']==age]
            size_df_temp = np.arange(len(df_temp))
            if len(size_df_temp) < size_inc:
                print('Warning Bobo Colluzzi VF group {} smaller than amount requested'.format(age))
            np.random.seed(42)
            np.random.shuffle(size_df_temp)
            index_df_temp_inc = size_df_temp[:size_inc]
            index_df_temp_not_inc = size_df_temp[size_inc:]
            df_train = pd.concat([df_train, df_temp.iloc[index_df_temp_inc]])
            df_test = pd.concat([df_test, df_temp.iloc[index_df_temp_not_inc]])

        print('Percentage of field mosquitoes inc {} - Num mosquitoes {} / {}'.format(valid_perc*100, len(df_train[df_train['RearCnd']=='VF']), len(df_vf)))
        print('Total number of mosquitoes in the Train set {}'.format(len(df_train)))
        
        X = df_train.iloc[:,6:-1]
        y_age = df_train["Age"]
        y_age_groups = df_train["AgeGroup"]
        y_species = df_train["Species"]
        y_status = df_train["Status"]
#         print('shape of X : {}'.format(X.shape))
#         print('shape of y age : {}'.format(y_age.shape))
#         print('shape of y age groups : {}'.format(y_age_groups.shape))
#         print('shape of y species : {}'.format(y_species.shape))
#         print('shape of y status : {}'.format(y_status.shape))
        self.X = np.asarray(X)
        y_age = np.asarray(y_age)
        self.y_age_groups = np.asarray(y_age_groups)
        self.y_species = np.asarray(y_species)
        y_status = np.asarray(y_status)

        X_vf = df_test.iloc[:,6:-1]
        y_age_vf = df_test["Age"]
        y_age_groups_vf = df_test["AgeGroup"]
        y_species_vf = df_test["Species"]
        y_status_vf = df_test["Status"]
#         print('shape of X_vf : {}'.format(X_vf.shape))
#         print('shape of y_age_vf age : {}'.format(y_age_vf.shape))
#         print('shape of y_age_groups_vf : {}'.format(y_age_groups_vf.shape))
#         print('shape of y y_species_vf : {}'.format(y_species_vf.shape))
#         print('shape of y y_status_vf : {}'.format(y_status_vf.shape))
        self.X_vf = np.asarray(X_vf)
        y_age_vf = np.asarray(y_age_vf)
        self.y_age_groups_vf = np.asarray(y_age_groups_vf)
        self.y_species_vf = np.asarray(y_species_vf)
        y_status_vf = np.asarray(y_status_vf)



In [5]:
def data_loader(valid_perc):
    return data_loader_class(valid_perc)

## Function used to create a new folder for the CNN outputs.
Useful to stop forgetting to name a new folder when trying out a new model varient and overwriting a days training.

In [6]:
def build_folder(fold, to_build = False):
    if not os.path.isdir(fold):
        if to_build == True:
            os.mkdir(fold)
        else:
            print('Directory does not exists, not creating directory!')
    else:
        if to_build == True:
            raise NameError('Directory already exists, cannot be created!')

## Function for plotting confusion matrcies
This normalizes the confusion matrix and ensures neat plotting for all outputs.

In [7]:
def plot_confusion_matrix(cm, classes, output, save_path, model_name, fold,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          printout=False):

    font = {'weight' : 'normal',
            'size'   : 18}

    matplotlib.rc('font', **font)

    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        if printout:
            print("Normalized confusion matrix")
    else:
        if printout:
            print('Confusion matrix, without normalization')

    if printout:
        print(cm)
    
    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=1) # np.max(np.sum(cm, axis=1)))
#     plt.title([title+' - '+model_name])
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout(pad=2)
#     plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig((save_path+"Confusion_Matrix_"+model_name+"_"+fold+"_"+output[1:]+".png"))
    plt.close()

## Function used for visualizing outputs
This splits the output data into the four categories before plotting the confusion matricies.

In [8]:
## for visualizing losses and metrics once the neural network fold is trained
def visualize(histories, save_path, model_name, fold, classes, outputs, predicted, true):
    # Sort out predictions and true labels
    for label_predictions_arr, label_true_arr, classes, outputs in zip(predicted, true, classes, outputs):
        classes_pred = np.argmax(label_predictions_arr, axis=-1)
        classes_true = np.argmax(label_true_arr, axis=-1)
        cnf_matrix = confusion_matrix(classes_true, classes_pred)
        plot_confusion_matrix(cnf_matrix, classes, outputs, save_path, model_name, fold)

## Data logging

In [9]:
## for logging data associated with the model
def log_data(log, name, fold, save_path):
    f = open((save_path+name+'_'+str(fold)+'_log.txt'), 'w')
    np.savetxt(f, log)
    f.close()

In [10]:
def save_obj(obj, name, savedir_main):
    with open(savedir_main + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [11]:
def load_obj(name, savedir_main):
    with open(savedir_main + name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Fucntion for graphing the training data
This fucntion creates tidy graphs of loss and accuracy as the models are training.

In [12]:
def graph_history(history, model_name, model_ver_num, fold, save_path):

    font = {'weight' : 'normal',
            'size'   : 18}

    matplotlib.rc('font', **font)
    
    #not_validation = list(filter(lambda x: x[0:3] != "val", history.history.keys()))
#     print('history.history.keys : {}'.format(history.history.keys()))
    filtered = filter(lambda x: x[0:3] != "val", history.history.keys())
    not_validation = list(filtered)
    for i in not_validation:
        plt.figure(figsize=(15,7))
#         plt.title(i+"/ "+"val_"+i)
        plt.plot(history.history[i], label=i)
        plt.plot(history.history["val_"+i], label="val_"+i)
        plt.legend()
        plt.xlabel("epoch")
        plt.ylabel(i)
        plt.savefig(save_path +model_name+"_"+str(model_ver_num)+"_"+str(fold)+"_"+i)
        plt.close()

## funciton to create the CNN
This function takes as an input a list of dictionaries. Each element in the list is a new hidden layer in the model. For each layer the dictionary defines the layer to be used.

### Available options are:
Convolutional Layer:
* type = 'c'
* filter = optional number of filters
* kernel = optional size of the filters
* stride = optional size of stride to take between filters
* pooling = optional width of the max pooling
* {'type':'c', 'filter':16, 'kernel':5, 'stride':1, 'pooling':2}

dense layer:
* type = 'd'
* width = option width of the layer
* {'type':'d', 'width':500}

In [14]:
def create_models(model_shape, input_layer):

    regConst = 0.02
    sgd = keras.optimizers.SGD(lr=0.003, decay=1e-5, momentum=0.9, nesterov=True, clipnorm=1.)
    cce = 'categorical_crossentropy'

    input_vec = Input(name='input', shape=(input_layer_dim,1))

    for i, layerwidth in zip(range(len(model_shape)),model_shape):
        if i == 0:
            if model_shape[i]['type'] == 'c':
                xd = Conv1D(name=('Conv'+str(i+1)), filters=model_shape[i]['filter'], 
                 kernel_size = model_shape[i]['kernel'], strides = model_shape[i]['stride'],
                 activation = 'relu',
                 kernel_regularizer=l2(regConst), 
                 kernel_initializer='he_normal')(input_vec)
                xd = BatchNormalization(name=('batchnorm_'+str(i+1)))(xd)
                xd = MaxPooling1D(pool_size=(model_shape[i]['pooling']))(xd)
                
            elif model_shape[i]['type'] == 'd':
                xd = Dense(name=('d'+str(i+1)), units=model_shape[i]['width'], activation='relu', 
                 kernel_regularizer=l2(regConst), 
                 kernel_initializer='he_normal')(input_vec)
                xd = BatchNormalization(name=('batchnorm_'+str(i+1)))(xd) 
                xd = Dropout(name=('dout'+str(i+1)), rate=0.5)(xd) 
                
        else:
            if model_shape[i]['type'] == 'c':
                xd = Conv1D(name=('Conv'+str(i+1)), filters=model_shape[i]['filter'], 
                 kernel_size = model_shape[i]['kernel'], strides = model_shape[i]['stride'],
                 activation = 'relu',
                 kernel_regularizer=l2(regConst), 
                 kernel_initializer='he_normal')(xd)
                xd = BatchNormalization(name=('batchnorm_'+str(i+1)))(xd)
                xd = MaxPooling1D(pool_size=(model_shape[i]['pooling']))(xd)
                
            elif model_shape[i]['type'] == 'd':
                if model_shape[i-1]['type'] == 'c':
                    xd = Flatten()(xd)
                    
                xd = Dropout(name=('dout'+str(i+1)), rate=0.5)(xd)
                xd = Dense(name=('d'+str(i+1)), units=model_shape[i]['width'], activation='relu', 
                 kernel_regularizer=l2(regConst), 
                 kernel_initializer='he_normal')(xd)
                xd = BatchNormalization(name=('batchnorm_'+str(i+1)))(xd) 
        
    
#     xAge     = Dense(name = 'age', units = 17, 
#                      activation = 'softmax', 
#                      kernel_regularizer = l2(regConst), 
#                      kernel_initializer = 'he_normal')(xd)
    xAgeGroup     = Dense(name = 'age_group', units = 3, 
                     activation = 'softmax', 
                     kernel_regularizer = l2(regConst), 
                     kernel_initializer = 'he_normal')(xd)
    xSpecies = Dense(name ='species', units = 3, 
                     activation = 'softmax', 
                     kernel_regularizer = l2(regConst), 
                     kernel_initializer = 'he_normal')(xd)

    outputs = []
#     for i in ['xAge', 'xAgeGroup', 'xSpecies']:
    for i in ['xAgeGroup', 'xSpecies']:
        outputs.append(locals()[i])
    model = Model(inputs = input_vec, outputs = outputs)
    
    model.compile(loss=cce, metrics=['acc'], 
                  optimizer=sgd)
    print(model.metrics)
#     model.summary()
    return model

## Function to train the model

This function will split the data into training and validation and call the create models function. This fucntion returns the model and training history.

In [15]:
def train_models(model_to_test, save_path, SelectFreqs=False):
    
    out_path = save_path+'out/'
    build_folder(out_path, True)

    model_shape = model_to_test["model_shape"][0]
    model_name = model_to_test["model_name"][0]
#     input_layer_dim = model_to_test["input_layer_dim"][0]
    model_ver_num = model_to_test["model_ver_num"][0]
    fold = model_to_test["fold"][0]
    label = model_to_test["labels"][0]
    features = model_to_test["features"][0]
    classes = model_to_test["classes"][0]
    outputs = model_to_test["outputs"][0]
    compile_loss = model_to_test["compile_loss"][0]
    compile_metrics = model_to_test["compile_metrics"][0]

#     ## Split into training / testing
#     test_splits = train_test_split(features, *(label), test_size=0.1, shuffle=True, random_state=rand_seed)
#     ## Pack up data
#     X_train = test_splits.pop(0)
#     X_val = test_splits.pop(0)
#     y_train = test_splits[::2]
#     y_val = test_splits[1::2]
    
#     out_model = create_models(model_shape, input_layer_dim, SelectFreqs)
#     out_model.summary()
#     out_history = out_model.fit(x = X_train, 
#                             y = y_train,
#                             batch_size = 128*16, 
#                             verbose = 0, 
#                             epochs = 8000,
#                             validation_data = (X_val, y_val),
#                             callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', 
#                                         patience=400, verbose=0, mode='auto'), 
#                                         CSVLogger(save_path+model_name+"_"+str(model_ver_num)+'.csv', append=True, separator=';')])
#     scores = out_model.evaluate(X_val, y_val)
#     print(out_model.metrics_names)

    ## Kfold training
    seed = rand_seed
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    ## Split data into test and train
    
    model_ver_num = 0
    cv_scores = []
    best_score = 0
    for train_index, val_index in kfold.split(features):
        print('Fold {} Running'.format(model_ver_num))
        
        X_train, X_val = features[train_index], features[val_index]
        y_train, y_val = list(map(lambda y:y[train_index], label)), list(map(lambda y:y[val_index], label))

        model = create_models(model_shape, input_layer_dim)
        if model_ver_num == 0:
            model.summary()

        history = model.fit(x = X_train, 
                            y = y_train,
                            batch_size = 128*16, 
                            verbose = 0, 
                            epochs = 8000,
                            validation_data = (X_val, y_val),
                            callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', 
                                        patience=400, verbose=0, mode='auto'), 
                                        CSVLogger(out_path+model_name+"_"+str(model_ver_num)+'.csv', append=True, separator=';')])
        scores = model.evaluate(X_val, y_val)
        model.save((out_path+model_name+"_"+str(model_ver_num)+'_Model.h5'))
        graph_history(history, model_name, model_ver_num, 0, out_path)
#         print(model.metrics_names)
#         print(scores)
        if (scores[3] + scores[4]) > best_score:
            out_model = model
            out_history = history
        
        model_ver_num += 1
        
#         # Clear the Keras session, otherwise it will keep adding new
#         # models to the same TensorFlow graph each time we create
#         # a model with a different set of hyper-parameters.
#         K.clear_session()

#         # Delete the Keras model with these hyper-parameters from memory.
#         del model
        
    out_model.save((save_path+'Best_Model.h5'))
    graph_history(out_history, 'Best_Model', 0, 0, save_path)
    
    return out_model, out_history

## Main section

Functionality:
* Oganises the data into a format of lists of data, classes, labels.
* Define the CNN to be built.
* Define the KFold validation to be used.
* Build a folder to output data into.
* Standardize and oragnise data into training/testing.
* Call the model training.
* Organize outputs and call visualization for plotting and graphing.


## 4 layers

In [16]:
## Name a folder for the outputs to go into
outdir = "Results_Paper/"
build_folder(outdir, False)

val_results = {'loss':[], 'age_group_loss':[], 'species_loss':[], 'age_group_acc':[], 'species_acc':[]}
histories = []

start_time = time()

# for valid_inc_perc in [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
for valid_inc_perc in [0]:
    
    savedir = (outdir+"TEST/")
    build_folder(savedir, True)

    data_extract = data_loader(valid_inc_perc)
    
    ## Input CNN Size
    input_layer_dim = len(data_extract.X[0])

    ## Transform Data
    y_age_groups_list = [[age] for age in data_extract.y_age_groups]
    y_species_list = [[species] for species in data_extract.y_species]
    age_groups = MultiLabelBinarizer().fit_transform(np.array(y_age_groups_list))
    age_group_classes = ["1-4", "5-10", "11-17"]
    species = MultiLabelBinarizer().fit_transform(np.array(y_species_list))
    species_classes = list(np.unique(y_species_list))
    y_age_groups_list_vf = [[age] for age in data_extract.y_age_groups_vf]
    y_species_list_vf = [[species] for species in data_extract.y_species_vf]
    age_groups_vf = MultiLabelBinarizer().fit_transform(np.array(y_age_groups_list_vf))
    species_vf = MultiLabelBinarizer().fit_transform(np.array(y_species_list_vf))

    ## Labels
    labels_default, classes_default, outputs_default = [age_groups, species], [age_group_classes, species_classes], ['xAgeGroup', 'xSpecies']
    labels_default_vf, classes_default_vf, outputs_default_vf = [age_groups_vf, species_vf], [age_group_classes, species_classes], ['xAgeGroup', 'xSpecies']

    ## Declare and train the model
    model_size = [{'type':'c', 'filter':16, 'kernel':8, 'stride':1, 'pooling':1},
                 {'type':'c', 'filter':16, 'kernel':8, 'stride':2, 'pooling':2},
                 {'type':'c', 'filter':16, 'kernel':3, 'stride':1, 'pooling':1},
                 {'type':'c', 'filter':16, 'kernel':6, 'stride':2, 'pooling':2},
    #              {'type':'c', 'filter':16, 'kernel':5, 'stride':1, 'pooling':2},
                 {'type':'d', 'width':400}]

    ## Name the model
    model_name = ('Valid_Inc_'+str(valid_inc_perc))

    ## Scale train, test
    scl = StandardScaler()
    features_scl = scl.fit(X=np.vstack((data_extract.X, data_extract.X_vf)))
    X_train = features_scl.transform(X=data_extract.X)
    X_test = features_scl.transform(X=data_extract.X_vf)

    ## Split data into test and train
    y_train, y_test = list(map(lambda y:y, labels_default)), list(map(lambda y:y, labels_default_vf))

    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)

    model_to_test = {
        "model_shape" : [model_size], # defines the hidden layers of the model
        "model_name"  : [model_name],
        "input_layer_dim"  : [input_layer_dim], # size of input layer
        "model_ver_num"  : [0],
        "fold"  : [0], # kf.split number on
        "labels"   : [y_train],
        "features" : [X_train],
        "classes"  : [classes_default],
        "outputs"   : [outputs_default],
        "compile_loss": [{'age': 'categorical_crossentropy'}],
        "compile_metrics" :[{'age': 'accuracy'}]
    }

    ## Call function to train all the models from the dictionary
    model, history = train_models(model_to_test, savedir)
    histories.append(history)

    predicted_labels = list([] for i in range(len(y_train)))
    true_labels = list([] for i in range(len(y_train)))

    y_predicted = model.predict(X_test)
    temp_eval = model.evaluate(X_test, y_test)
    for metric, res in zip(model.metrics_names, temp_eval):
        val_results[metric].append(res)
    print(val_results)

    predicted_labels = [x+[y] for x,y in zip(predicted_labels,y_predicted)]
    true_labels = [x+[y] for x,y in zip(true_labels,y_test)]

    predicted_labels = [predicted_labels[i][0].tolist() for i in range(len(predicted_labels))]
    true_labels = [true_labels[i][0].tolist() for i in range(len(true_labels))]

    ## Visualize the results
    visualize(histories, savedir, model_name, "0", classes_default, outputs_default, predicted_labels, true_labels)

    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()

    # Delete the Keras model with these hyper-parameters from memory.
    del model

end_time = time()
print('Run time : {} s'.format(end_time-start_time))
print('Run time : {} m'.format((end_time-start_time)/60))
print('Run time : {} h'.format((end_time-start_time)/3600))

# save_obj(val_results, 'Validation_Results_Dict_4_layers', savedir_main)


Percentage of field mosquitoes inc 0 - Num mosquitoes 0 / 3277
Total number of mosquitoes in the Train set 7200
Fold 0 Running
['acc']
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 1625, 1)      0                                            
__________________________________________________________________________________________________
Conv1 (Conv1D)                  (None, 1618, 16)     144         input[0][0]                      
__________________________________________________________________________________________________
batchnorm_1 (BatchNormalization (None, 1618, 16)     64          Conv1[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 1618, 16)     0           batchnor

KeyboardInterrupt: 

In [ ]:
# input_layer_dim = len(X[0])

# # y_age_groups = np.where((y_age<=4), 0, 0)
# # y_age_groups = np.where((y_age>=5) & (y_age<=10), 1, y_age_groups)
# # y_age_groups = np.where((y_age>=11), 2, y_age_groups)

# ## Ages transformed
# # y_age_list = [[age] for age in y_age]
# y_age_groups_list = [[age] for age in y_age_groups]
# y_species_list = [[species] for species in y_species]
# y_status_list = [[status] for status in y_status]
# # age = MultiLabelBinarizer().fit_transform(np.array(y_age_list))
# # age_classes = list(np.unique(y_age_list))
# age_groups = MultiLabelBinarizer().fit_transform(np.array(y_age_groups_list))
# age_group_classes = ["1-4", "5-10", "11-17"]
# species = MultiLabelBinarizer().fit_transform(np.array(y_species_list))
# species_classes = list(np.unique(y_species_list))
# status = MultiLabelBinarizer().fit_transform(np.array(y_status_list))
# status_classes = list(np.unique(y_status_list))

# outdir = "Results_Paper/"
# build_folder(outdir, False)
# SelectFreqs = False

# ## Labels default - all classification
# # labels_default, classes_default, outputs_default = [age, age_groups, species], [age_classes, age_group_classes, species_classes], ['xAge', 'xAgeGroup', 'xSpecies']
# labels_default, classes_default, outputs_default = [age_groups, species], [age_group_classes, species_classes], ['xAgeGroup', 'xSpecies']


# ## Declare and train the model
# model_size = [{'type':'c', 'filter':12, 'kernel':8, 'stride':1, 'pooling':1},
#              {'type':'c', 'filter':12, 'kernel':8, 'stride':2, 'pooling':2},
#              {'type':'c', 'filter':12, 'kernel':3, 'stride':2, 'pooling':2},
# #              {'type':'c', 'filter':16, 'kernel':6, 'stride':2, 'pooling':2},
# #              {'type':'c', 'filter':16, 'kernel':5, 'stride':1, 'pooling':2},
#              {'type':'d', 'width':250}]

# ## Name the model
# model_name = 'Baseline_CNN'
# label = labels_default
    
# ## Split data into 10 folds for training/testing
# kf = KFold(n_splits=10, shuffle=True, random_state=rand_seed)
            
# ## Features
# # features = X
    
# histories = []
# fold = 1
# train_model = True

# ## Name a folder for the outputs to go into
# savedir = (outdir+"Trian_3_layers/")
# build_folder(savedir, True)
    
# start_time = time()
# save_predicted = []
# save_true = []

# ## Scale train, test
# scl = StandardScaler()
# features_scl = scl.fit(X=X)
# features = features_scl.transform(X=X)

# ## Split into training / testing
# test_splits = train_test_split(features, *(label), test_size=0.1, shuffle=True, random_state=rand_seed)
# ## Pack up data
# X_train = test_splits.pop(0)
# X_test = test_splits.pop(0)
# y_train = test_splits[::2]
# y_test = test_splits[1::2]

# if not SelectFreqs:
#     X_train = np.expand_dims(X_train, axis=2)
#     X_test = np.expand_dims(X_test, axis=2)


# model_to_test = {
#     "model_shape" : [model_size], # defines the hidden layers of the model
#     "model_name"  : [model_name],
#     "input_layer_dim"  : [input_layer_dim], # size of input layer
#     "model_ver_num"  : [0],
#     "fold"  : [fold], # kf.split number on
#     "labels"   : [y_train],
#     "features" : [X_train],
#     "classes"  : [classes_default],
#     "outputs"   : [outputs_default],
#     "compile_loss": [{'age': 'categorical_crossentropy'}],
#     "compile_metrics" :[{'age': 'accuracy'}]
# }

# ## Call function to train all the models from the dictionary
# model, history = train_models(model_to_test, savedir, SelectFreqs=SelectFreqs)
# histories.append(history)

# predicted_labels = list([] for i in range(len(y_train)))
# true_labels = list([] for i in range(len(y_train)))

# y_predicted = model.predict(X_test)

# predicted_labels = [x+[y] for x,y in zip(predicted_labels,y_predicted)]
# true_labels = [x+[y] for x,y in zip(true_labels,y_test)]

# predicted_labels = [predicted_labels[i][0].tolist() for i in range(len(predicted_labels))]
# true_labels = [true_labels[i][0].tolist() for i in range(len(true_labels))]

# for pred, tru in zip(predicted_labels, true_labels):
#     save_predicted.append(pred)
#     save_true.append(tru)

# ## Visualize the results
# visualize(histories, savedir, model_name, str(fold), classes_default, outputs_default, predicted_labels, true_labels)
# # log_data(test_index, 'test_index', fold, savedir)

# # Clear the Keras session, otherwise it will keep adding new
# # models to the same TensorFlow graph each time we create
# # a model with a different set of hyper-parameters.
# K.clear_session()

# # Delete the Keras model with these hyper-parameters from memory.
# del model

# # visualize(1, savedir, model_name, "Averaged", classes_default, outputs_default, save_predicted, save_true)
# end_time = time()
# print('Run time : {} s'.format(end_time-start_time))
# print('Run time : {} m'.format((end_time-start_time)/60))
# print('Run time : {} h'.format((end_time-start_time)/3600))


In [9]:
font = {'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

## Size / Accuracy Age / Accuracy Species
# size_results = [[200952, 395256, 641878, 1594506],[33,32.7,34.7,32.7],[38.3,38,37.7,38.7]]
size_results = [[201, 395, 642, 1595],[33,32.7,34.7,32.7],[38.3,38,37.7,38.7]]

plt.figure(figsize=(15,7))
plt.scatter(size_results[0], size_results[1], s=80, c='b', label='Grouped Age Accuracy')
plt.scatter(size_results[0], size_results[2], s=80, c='g', label='Species Accuracy')
z = np.polyfit(size_results[0], size_results[1], 1)
p = np.poly1d(z)
plt.plot(size_results[0],p(size_results[0]),"b--")
z = np.polyfit(size_results[0], size_results[2], 1)
p = np.poly1d(z)
plt.plot(size_results[0],p(size_results[0]),"g--")
plt.legend()
plt.xlabel("Number of Trainable Parameters (Thousands)")
plt.ylabel('Accuracy')
plt.xlim([150,1650])
plt.savefig('Results_Paper/Molel_Size_Study/Model_Size_Study_Results.png')
plt.close()

## Re-doing model size reduction plot using accuracy predicted by the model not the confusion matrices.

Using confusion matrices doesn't work as it assumes a balanced number of each class in the test set where in reality this is not the case and each accuracy needs scaling by percentage of total test samples

In [35]:
val_results = {'params':[], 'loss':[], 'age_group_loss':[], 'species_loss':[], 'age_group_acc':[], 'species_acc':[]}

for model_name in ['Train_5_layers/Valid_Inc_0_Model.h5', 'Train_4_layers/Best_Model.h5',
                  'Train_3_layers/Valid_Inc_0_Model.h5', 'Train_2_layers/Valid_Inc_0_Model.h5']:

    model = load_model(('/home/josh/Documents/Mosquito_Project/MIMI-Analysis/Models/Neural_Networks/CNN/Paper/Results_Paper/Molel_Size_Study/'+model_name))

    valid_inc_perc =0

    data_extract = data_loader(valid_inc_perc)

    ## Input CNN Size
    input_layer_dim = len(data_extract.X[0])

    ## Transform Data
    y_age_groups_list = [[age] for age in data_extract.y_age_groups]
    y_species_list = [[species] for species in data_extract.y_species]
    age_groups = MultiLabelBinarizer().fit_transform(np.array(y_age_groups_list))
    age_group_classes = ["1-4", "5-10", "11-17"]
    species = MultiLabelBinarizer().fit_transform(np.array(y_species_list))
    species_classes = list(np.unique(y_species_list))
    y_age_groups_list_vf = [[age] for age in data_extract.y_age_groups_vf]
    y_species_list_vf = [[species] for species in data_extract.y_species_vf]
    age_groups_vf = MultiLabelBinarizer().fit_transform(np.array(y_age_groups_list_vf))
    species_vf = MultiLabelBinarizer().fit_transform(np.array(y_species_list_vf))

    ## Labels
    labels_default, classes_default, outputs_default = [age_groups, species], [age_group_classes, species_classes], ['xAgeGroup', 'xSpecies']
    labels_default_vf, classes_default_vf, outputs_default_vf = [age_groups_vf, species_vf], [age_group_classes, species_classes], ['xAgeGroup', 'xSpecies']


    ## Scale train, test
    scl = StandardScaler()
    features_scl = scl.fit(X=np.vstack((data_extract.X, data_extract.X_vf)))
    X_train = features_scl.transform(X=data_extract.X)
    X_test = features_scl.transform(X=data_extract.X_vf)

    ## Split data into test and train
    y_train, y_test = list(map(lambda y:y, labels_default)), list(map(lambda y:y, labels_default_vf))

    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)


    predicted_labels = list([] for i in range(len(y_train)))
    true_labels = list([] for i in range(len(y_train)))
    
    trainable_count = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    val_results['params'].append(trainable_count)
    
    y_predicted = model.predict(X_test)
    temp_eval = model.evaluate(X_test, y_test)
    for metric, res in zip(model.metrics_names, temp_eval):
        val_results[metric].append(res)

print(val_results)


Percentage of field mosquitoes inc 0 - Num mosquitoes 0 / 3277
Total number of mosquitoes in the Train set 7200
3277/3277 [==============================] - 1s 328us/step
Percentage of field mosquitoes inc 0 - Num mosquitoes 0 / 3277
Total number of mosquitoes in the Train set 7200
3277/3277 [==============================] - 1s 337us/step
Percentage of field mosquitoes inc 0 - Num mosquitoes 0 / 3277
Total number of mosquitoes in the Train set 7200
3277/3277 [==============================] - 1s 401us/step
Percentage of field mosquitoes inc 0 - Num mosquitoes 0 / 3277
Total number of mosquitoes in the Train set 7200
3277/3277 [==============================] - 1s 397us/step
{'params': [1594506, 641878, 395256, 200952], 'loss': [4.942620224002516, 4.452852591822196, 3.6912602784613995, 3.7235344156172716], 'age_group_loss': [1.9838111671610024, 1.8465644805488934, 1.6836907164275254, 1.5485919786956495], 'species_loss': [2.8875805031818556, 2.4992994878089823, 1.8783734314937144, 1.979

In [33]:
print(y_test[1].shape)
num_AA = (np.unique(y_test[1][:,0], return_counts=True))[1][1]
num_AC = (np.unique(y_test[1][:,1], return_counts=True))[1][1]
num_AG = (np.unique(y_test[1][:,2], return_counts=True))[1][1]
num_tot = num_AA + num_AC + num_AG
print(f'perc AA in test : {num_AA/num_tot}')
print(f'perc AC in test : {num_AC/num_tot}')
print(f'perc AG in test : {num_AG/num_tot}')

(3277, 3)
perc AA in test : 0.4107415318889228
perc AC in test : 0.07567897467195606
perc AG in test : 0.5135794934391211


In [40]:
font = {'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)


## Size / Accuracy Age / Accuracy Species
# size_results = [[200952, 395256, 641878, 1594506],[33,32.7,34.7,32.7],[38.3,38,37.7,38.7]]
size_results = [[201, 395, 642, 1595],[33,32.7,34.7,32.7],[38.3,38,37.7,38.7]]

plt.figure(figsize=(15,7))
plt.scatter([val/1000 for val in val_results['params']], [val*100 for val in val_results['age_group_acc']], s=80, c='b', label='Grouped Age Accuracy')
plt.scatter([val/1000 for val in val_results['params']], [val*100 for val in val_results['species_acc']], s=80, c='g', label='Species Accuracy')
z = np.polyfit([val/1000 for val in val_results['params']], [val*100 for val in val_results['age_group_acc']], 1)
p = np.poly1d(z)
plt.plot([val/1000 for val in val_results['params']],p([val/1000 for val in val_results['params']]),"b--")
z = np.polyfit([val/1000 for val in val_results['params']], [val*100 for val in val_results['species_acc']], 1)
p = np.poly1d(z)
plt.plot([val/1000 for val in val_results['params']],p([val/1000 for val in val_results['params']]),"g--")
plt.legend()
plt.xlabel("Number of Trainable Parameters (Thousands)")
plt.ylabel('Accuracy')
plt.xlim([150,1650])
plt.savefig('Results_Paper/Molel_Size_Study/Model_Size_Study_Results_v2.png')
plt.close()